### Battle of the Neighborhoods

## This Notebook will be used for the IBM Data Science certificate


In [1]:
import numpy as np
import pandas as pd
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
